<a href="https://colab.research.google.com/github/PierfrancescoLindia/Dubai-Properties-RIDGE-LASSO-AND-ELASTIC-NET-REGRESSION/blob/main/Qwen2_5_7B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# STEP 1: Installazione delle librerie
# Questo passaggio prepara l'ambiente Colab per il fine-tuning veloce.
# Potrebbe impiegare circa 2-3 minuti.

%%capture
!pip install unsloth
# Scarichiamo le dipendenze specifiche per Colab
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install unsloth[colab-new]

print("INSTALLAZIONE COMPLETATA! Possiamo procedere.")

In [ ]:
# STEP 2: Caricamento del Modello Qwen 2.5 7B
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Il modello ricorderà conversazioni lunghe fino a 2048 parole/token
dtype = None # Lasciamo decidere a Unsloth il tipo di dati migliore
load_in_4bit = True # Fondamentale: usa 4-bit per non finire la memoria RAM

print("Inizio il download del modello... potrebbe impiegare 1-2 minuti.")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-7B-Instruct-bnb-4bit", # Versione ottimizzata
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Aggiungiamo gli adattatori LoRA (i "moduli" che addestreremo)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Potenza di apprendimento (16 è standard ed efficiente)
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

print("MODELLO CARICATO CON SUCCESSO! Pronto per i dati.")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Inizio il download del modello... potrebbe impiegare 1-2 minuti.
==((====))==  Unsloth 2025.11.6: Fast Qwen2 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# STEP 3: Creazione e Preparazione del Dataset (CORRETTO)
import json
from datasets import load_dataset

# 1. Definiamo il System Prompt
system_prompt = """Sei un assistente AI per la gestione appuntamenti del barbiere.
Il tuo compito è analizzare il messaggio dell'utente ed estrarre le informazioni in formato JSON.
Rispondi SOLO con un blocco JSON valido che segua questo schema:
{
  "intent": "book | reschedule | cancel | ask_info",
  "date": "YYYY-MM-DD",
  "time": "HH:MM",
  "service": "taglio | barba | taglio+barba",
  "notes": "eventuali note o null"
}
"""

# 2. Creiamo la lista degli esempi (Dataset Minimo)
# CORREZIONE: In Python si usa 'None' invece di 'null'.
# json.dumps convertirà automaticamente 'None' in 'null' nel file finale.
data_examples = [
    {
        "instruction": system_prompt,
        "input": "[DATA CORRENTE: 2023-10-02 (Lunedì)] Ciao, vorrei prenotare un taglio per domani alle 15.",
        "output": json.dumps({"intent": "book", "date": "2023-10-03", "time": "15:00", "service": "taglio", "notes": None})
    },
    {
        "instruction": system_prompt,
        "input": "[DATA CORRENTE: 2023-11-10] Vorrei fare la barba sabato prossimo alle 10.",
        "output": json.dumps({"intent": "book", "date": "2023-11-18", "time": "10:00", "service": "barba", "notes": None})
    },
    {
        "instruction": system_prompt,
        "input": "[DATA CORRENTE: 2024-01-05] Devo annullare l'appuntamento di domani.",
        "output": json.dumps({"intent": "cancel", "date": "2024-01-06", "time": None, "service": None, "notes": None})
    },
    {
        "instruction": system_prompt,
        "input": "[DATA CORRENTE: 2024-02-20] Quanto costa il taglio?",
        "output": json.dumps({"intent": "ask_info", "date": None, "time": None, "service": "taglio", "notes": "Quanto costa il taglio?"})
    },
    {
        "instruction": system_prompt,
        "input": "[DATA CORRENTE: 2023-10-01] Sposta il mio appuntamento a giovedì alle 17:30.",
        "output": json.dumps({"intent": "reschedule", "date": "2023-10-05", "time": "17:30", "service": None, "notes": None})
    }
]

# 3. Scriviamo il file su disco
with open('dataset.jsonl', 'w') as f:
    for entry in data_examples:
        f.write(json.dumps(entry) + '\n')

# 4. Funzione per formattare i dati
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# 5. Carichiamo il dataset
dataset = load_dataset("json", data_files="dataset.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

print("DATASET PREPARATO! Esempi caricati:", len(dataset))

In [6]:
# STEP 4: Configurazione e Avvio del Training (Versione NO WANDB)
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

print("Inizio configurazione del Trainer...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = 60,
        warmup_steps = 5,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # <--- QUESTA RIGA BLOCCA LA RICHIESTA DI WANDB
    ),
)

print("Avvio addestramento... (Questa volta partirà subito!)")
trainer_stats = trainer.train()

Inizio configurazione del Trainer...


num_proc must be <= 5. Reducing num_proc to 5 for dataset of size 5.
The model is already on multiple devices. Skipping the move to device specified in `args`.


Avvio addestramento... (Questa volta partirà subito!)


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 60 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.832100
2,1.832100
3,1.746700
4,1.499400
5,1.285600
6,1.096600
7,0.909600
8,0.731900
9,0.558400
10,0.401700


In [7]:
# STEP 5: Test del Modello (Inference)
FastLanguageModel.for_inference(model) # Attiva la modalità veloce

# 1. Simuliamo una richiesta nuova (mai vista nel training)
messaggio_utente = "Ciao! Vorrei prenotare un taglio per dopodomani verso le 16."
data_oggi = "2023-12-01" # Supponiamo sia Venerdì 1 Dicembre

# 2. Prepariamo il prompt esattamente come nel training
prompt_input = alpaca_prompt.format(
    system_prompt, # L'istruzione sul formato JSON
    f"[DATA CORRENTE: {data_oggi}] {messaggio_utente}", # Input
    "", # Output lasciato vuoto per farlo riempire al modello
)

# 3. Generiamo la risposta
inputs = tokenizer([prompt_input], return_tensors = "pt").to("cuda")

# Limitiamo l'output a 128 token (il JSON è breve)
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)

# 4. Decodifichiamo e stampiamo solo la parte nuova
output_text = tokenizer.batch_decode(outputs)[0]

# Puliamo un po' l'output per mostrarti solo la risposta dell'AI
print("--- RISPOSTA DEL MODELLO ---")
print(output_text.split("### Response:")[1].strip().replace("<|endoftext|>", ""))

--- RISPOSTA DEL MODELLO ---
{"intent": "book", "date": "2023-12-03", "time": "16:00", "service": "taglio", "notes": null}<|im_end|>


In [2]:
# STEP 3 (PRO): Dataset Esteso (~60 Esempi)
import json
from datasets import load_dataset

# 1. System Prompt (Invariato)
system_prompt = """Sei un assistente AI per la gestione appuntamenti del barbiere.
Il tuo compito è analizzare il messaggio dell'utente ed estrarre le informazioni in formato JSON.
Rispondi SOLO con un blocco JSON valido che segua questo schema:
{
  "intent": "book | reschedule | cancel | ask_info",
  "date": "YYYY-MM-DD",
  "time": "HH:MM",
  "service": "taglio | barba | taglio+barba",
  "notes": "eventuali note o null"
}
"""

# 2. Lista dati estesa
data_examples = [
    # --- PRENOTAZIONI COMPLETE ---
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-10-02] Ciao, vorrei un taglio per domani alle 15.", "output": json.dumps({"intent": "book", "date": "2023-10-03", "time": "15:00", "service": "taglio", "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-10-02] Prenotami barba e capelli per il 5 ottobre alle 10.", "output": json.dumps({"intent": "book", "date": "2023-10-05", "time": "10:00", "service": "taglio+barba", "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-12-01] Vorrei venire venerdì prossimo alle 17:00 per la barba.", "output": json.dumps({"intent": "book", "date": "2023-12-08", "time": "17:00", "service": "barba", "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2024-01-10] Segnami un taglio per oggi alle 18:30.", "output": json.dumps({"intent": "book", "date": "2024-01-10", "time": "18:30", "service": "taglio", "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-05-20] Ciao, taglio alle 9 di domani mattina.", "output": json.dumps({"intent": "book", "date": "2023-05-21", "time": "09:00", "service": "taglio", "notes": None})},

    # --- PRENOTAZIONI INCOMPLETE (Il modello deve capire cosa manca) ---
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-10-02] Vorrei prenotare un taglio.", "output": json.dumps({"intent": "book", "date": None, "time": None, "service": "taglio", "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-10-02] Avete posto domani?", "output": json.dumps({"intent": "book", "date": "2023-10-03", "time": None, "service": None, "notes": "Avete posto domani?"})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-10-02] Vorrei venire alle 15.", "output": json.dumps({"intent": "book", "date": None, "time": "15:00", "service": None, "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-11-15] Devo fare la barba.", "output": json.dumps({"intent": "book", "date": None, "time": None, "service": "barba", "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2024-02-01] Vorrei prenotare.", "output": json.dumps({"intent": "book", "date": None, "time": None, "service": None, "notes": None})},

    # --- CANCELLAZIONI ---
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-09-12] Devo annullare l'appuntamento di domani.", "output": json.dumps({"intent": "cancel", "date": "2023-09-13", "time": None, "service": None, "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-09-12] Cancella la mia prenotazione del 15 settembre.", "output": json.dumps({"intent": "cancel", "date": "2023-09-15", "time": None, "service": None, "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-09-12] Non posso più venire oggi.", "output": json.dumps({"intent": "cancel", "date": "2023-09-12", "time": None, "service": None, "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-12-20] Elimina l'appuntamento di Marco.", "output": json.dumps({"intent": "cancel", "date": None, "time": None, "service": None, "notes": "Elimina appuntamento Marco"})},

    # --- SPOSTAMENTI (RESCHEDULE) ---
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-10-01] Puoi spostare il mio appuntamento a giovedì alle 17:30?", "output": json.dumps({"intent": "reschedule", "date": "2023-10-05", "time": "17:30", "service": None, "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-10-01] Vorrei cambiare orario, facciamo alle 16?", "output": json.dumps({"intent": "reschedule", "date": None, "time": "16:00", "service": None, "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-06-10] Rimanda il taglio a domani.", "output": json.dumps({"intent": "reschedule", "date": "2023-06-11", "time": None, "service": "taglio", "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-08-01] Posso spostare l'appuntamento al 5 agosto?", "output": json.dumps({"intent": "reschedule", "date": "2023-08-05", "time": None, "service": None, "notes": None})},

    # --- RICHIESTA INFO (ASK_INFO) ---
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2024-02-20] Quanto costa il taglio?", "output": json.dumps({"intent": "ask_info", "date": None, "time": None, "service": "taglio", "notes": "Quanto costa il taglio?"})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2024-02-20] A che ora chiudete?", "output": json.dumps({"intent": "ask_info", "date": None, "time": None, "service": None, "notes": "A che ora chiudete?"})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2024-02-20] Siete aperti il lunedì?", "output": json.dumps({"intent": "ask_info", "date": None, "time": None, "service": None, "notes": "Siete aperti lunedì?"})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2024-02-20] Fate anche la barba con panno caldo?", "output": json.dumps({"intent": "ask_info", "date": None, "time": None, "service": "barba", "notes": "Fate barba panno caldo?"})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2024-02-20] Dove vi trovate?", "output": json.dumps({"intent": "ask_info", "date": None, "time": None, "service": None, "notes": "Indirizzo?"})},

    # --- VARIE ED EVENTUALI (Edge Cases) ---
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-01-01] Buon anno! Vorrei prenotare per il 3 gennaio.", "output": json.dumps({"intent": "book", "date": "2023-01-03", "time": None, "service": None, "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-05-05] Ciao, sono Luca. Taglio martedì alle 18.", "output": json.dumps({"intent": "book", "date": "2023-05-09", "time": "18:00", "service": "taglio", "notes": "Cliente: Luca"})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-11-01] Grazie mille, a domani!", "output": json.dumps({"intent": "ask_info", "date": None, "time": None, "service": None, "notes": "Saluti"})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-07-20] No, intendevo cancellare quello di oggi.", "output": json.dumps({"intent": "cancel", "date": "2023-07-20", "time": None, "service": None, "notes": None})},
    {"instruction": system_prompt, "input": "[DATA CORRENTE: 2023-04-10] Vorrei prenotare barba e capelli per me e mio figlio domani alle 15.", "output": json.dumps({"intent": "book", "date": "2023-04-11", "time": "15:00", "service": "taglio+barba", "notes": "Per 2 persone (me e figlio)"})}
]

# Aggiungiamo un po' di duplicati per rinforzare i concetti (Tecnica comune se hai pochi dati manuali)
data_examples = data_examples * 2 # Ora abbiamo circa 50-60 esempi

# 3. Scrittura file
with open('dataset.jsonl', 'w') as f:
    for entry in data_examples:
        f.write(json.dumps(entry) + '\n')

# 4. Formattazione
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

dataset = load_dataset("json", data_files="dataset.jsonl", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

print(f"DATASET PRO CARICATO! Numero totale esempi: {len(dataset)}")

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/56 [00:00<?, ? examples/s]

DATASET PRO CARICATO! Numero totale esempi: 56


In [3]:
# STEP 4: Configurazione e Avvio del Training (Versione PRO)
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

print("Inizio configurazione del Trainer...")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        # AUMENTATO A 120 (per assicurarci che impari bene i nuovi dati)
        max_steps = 120,
        warmup_steps = 10,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Blocco WandB per evitare richieste di login
    ),
)

print("Avvio addestramento SERIO... Ci metterà circa 5-8 minuti.")
trainer_stats = trainer.train()

Inizio configurazione del Trainer...


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/56 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Avvio addestramento SERIO... Ci metterà circa 5-8 minuti.


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 56 | Num Epochs = 18 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 40,370,176 of 7,655,986,688 (0.53% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.913300
2,1.928900
3,1.909900
4,1.834100
5,1.610000
6,1.382900
7,1.333700
8,1.193100
9,0.969900
10,0.834100


In [4]:
# STEP 5: Test del Modello PRO (Inference)
FastLanguageModel.for_inference(model) # Attiva la modalità veloce

# Simuliamo una richiesta: Utente chiede "Lunedì prossimo" partendo da un Mercoledì
messaggio_utente = "Ciao, vorrei tagliare i capelli lunedì prossimo alle 10."
data_oggi = "2023-11-01" # Supponiamo sia Mercoledì 1 Novembre

# Prepariamo il prompt
prompt_input = alpaca_prompt.format(
    system_prompt,
    f"[DATA CORRENTE: {data_oggi}] {messaggio_utente}",
    "",
)

# Generiamo la risposta
inputs = tokenizer([prompt_input], return_tensors = "pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
output_text = tokenizer.batch_decode(outputs)[0]

# Stampiamo il risultato
print("--- RISPOSTA DEL NUOVO MODELLO ---")
print(output_text.split("### Response:")[1].strip().replace("<|endoftext|>", "").replace("<|im_end|>", ""))

--- RISPOSTA DEL NUOVO MODELLO ---
{"intent": "book", "date": "2023-11-06", "time": "10:00", "service": "taglio", "notes": null}


In [ ]:
# STEP 6: Salvataggio in formato GGUF (Per Ollama)
# Questo processo convertirà i pesi e li salverà in un file unico.
# Ci metterà circa 5-10 minuti.

print("Inizio conversione in GGUF... mettiti comodo.")

model.save_pretrained_gguf(
    "model_barbiere_pro", # Nome della cartella
    tokenizer,
    quantization_method = "q4_k_m" # Ottimizzato per Ollama
)

print("CONVERSIONE COMPLETATA!")
print("Adesso guarda nel menu a sinistra (icona cartella).")
print("Troverai la cartella 'model_barbiere_pro' e dentro il file .gguf")

Inizio conversione in GGUF... mettiti comodo.
Unsloth: Merging model weights to 16-bit format...


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files:  25%|██▌       | 1/4 [01:43<05:10, 103.47s/it]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]